In [1]:
import numpy as np
from tensorflow import keras
import tensorflow as tf
from keras.callbacks import ModelCheckpoint, EarlyStopping
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split

2024-03-11 14:30:44.046048: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-11 14:30:44.134700: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-11 14:30:44.137758: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-03-11 14:30:44.137770: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudar

In [2]:
data_dir = "./EuroSAT_RGB"
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(64, 64),
  batch_size=32)

Found 27000 files belonging to 10 classes.
Using 21600 files for training.


2024-03-11 14:30:46.343763: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-03-11 14:30:46.343892: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-03-11 14:30:46.343923: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2024-03-11 14:30:46.343953: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2024-03-11 14:30:46.343971: W tensorflow/c

In [3]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(64, 64),
  batch_size=32)

Found 27000 files belonging to 10 classes.
Using 5400 files for validation.


In [4]:
class_names = train_ds.class_names
print(class_names)

['AnnualCrop', 'Forest', 'HerbaceousVegetation', 'Highway', 'Industrial', 'Pasture', 'PermanentCrop', 'Residential', 'River', 'SeaLake']


In [5]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

(32, 64, 64, 3)
(32,)


In [6]:
normalization_layer = tf.keras.layers.Rescaling(1./255)
cropper_layer = tf.keras.layers.Cropping2D(cropping=((16, 16)))

cropped_ds = train_ds.map(lambda x, y: (cropper_layer(x), y))
normalized_ds_train = cropped_ds.map(lambda x, y: (normalization_layer(x), y))

cropped_ds = val_ds.map(lambda x, y: (cropper_layer(x), y))
normalized_ds_valid = cropped_ds.map(lambda x, y: (normalization_layer(x), y))

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [7]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = normalized_ds_train.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = normalized_ds_valid.cache().prefetch(buffer_size=AUTOTUNE)

In [8]:
model = keras.models.Sequential([
    keras.layers.Input((32, 32, 3)),
    keras.layers.Conv2D(32, (3, 3), activation='relu'), # 13 spectral bands
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Dropout(0.2),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Dropout(0.2),
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(10) # 10 different classes in the EuroSAT dataset
])

In [9]:
model.compile(optimizer='adam',
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [10]:
earlyStopping = EarlyStopping(monitor='val_accuracy', patience=10, verbose=0, mode='auto')
mcp_save = ModelCheckpoint("ground_finder.keras", save_best_only=True, monitor='val_accuracy', mode='auto')

In [11]:
# Train the model
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=100,
callbacks=[earlyStopping, mcp_save]
)

Epoch 1/100
675/675 [==============================] - 4s 5ms/step - loss: 1.4673 - accuracy: 0.4287 - val_loss: 1.1593 - val_accuracy: 0.5752
Epoch 2/100
675/675 [==============================] - 3s 4ms/step - loss: 1.0761 - accuracy: 0.6075 - val_loss: 1.2589 - val_accuracy: 0.5198
Epoch 3/100
675/675 [==============================] - 3s 4ms/step - loss: 0.9278 - accuracy: 0.6684 - val_loss: 1.2301 - val_accuracy: 0.5422
Epoch 4/100
675/675 [==============================] - 3s 4ms/step - loss: 0.8332 - accuracy: 0.6986 - val_loss: 1.0934 - val_accuracy: 0.6050
Epoch 5/100
675/675 [==============================] - 3s 4ms/step - loss: 0.7740 - accuracy: 0.7242 - val_loss: 0.9444 - val_accuracy: 0.6561
Epoch 6/100
675/675 [==============================] - 3s 4ms/step - loss: 0.7317 - accuracy: 0.7410 - val_loss: 0.8879 - val_accuracy: 0.6794
Epoch 7/100
675/675 [==============================] - 3s 4ms/step - loss: 0.6957 - accuracy: 0.7531 - val_loss: 0.9439 - val_accuracy: 0.6559